In [2]:
import kagglehub

path = kagglehub.dataset_download("gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/bitcoin-tweets-16m-tweets-with-sentiment-tagged


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 kB 27.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0


In [4]:
ls /root/.cache/kagglehub/datasets/gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged/versions/1

ls: cannot access '/root/.cache/kagglehub/datasets/gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged/versions/1': No such file or directory


In [5]:
from datasets import load_dataset

ds = load_dataset("edaschau/bitcoin_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/695 [00:00<?, ?B/s]

BTC_match_text.csv:   0%|          | 0.00/495M [00:00<?, ?B/s]

BTC_match_title.csv:   0%|          | 0.00/116M [00:00<?, ?B/s]

BTC_yahoo.csv:   0%|          | 0.00/396M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/210832 [00:00<?, ? examples/s]

In [53]:
pd

<module 'pandas' from '/usr/local/lib/python3.11/dist-packages/pandas/__init__.py'>

In [14]:
import pandas as pd
from datasets import load_dataset
import os
import numpy as np
from datetime import timedelta
from tqdm.notebook import tqdm
import yfinance as yf

print("Downloading price data for Bitcoin, Gold, and Oil...")
data = yf.download('BTC-USD GC=F CL=F', start='2014-01-01', end='2023-01-31', progress=False)
df_prices = pd.DataFrame(data['Close']['BTC-USD']).rename(columns={'BTC-USD': 'Close'})
df_gold = pd.DataFrame(data['Close']['GC=F']).rename(columns={'GC=F': 'Close'})
df_oil = pd.DataFrame(data['Close']['CL=F']).rename(columns={'CL=F': 'Close'})
df_prices.index = df_prices.index.tz_localize('UTC')
df_gold.index = df_gold.index.tz_localize('UTC')
df_oil.index = df_oil.index.tz_localize('UTC')
print("✅ Price data loaded.")

print("\nLoading LLM-Augmented Bitcoin dataset from Hugging Face...")
try:
    ds_llm = load_dataset("danilocorsi/LLMs-Sentiment-Augmented-Bitcoin-Dataset")
    df_llm_metrics = ds_llm['train'].to_pandas()
    # FIX: Removed unit='s' to correctly parse the date string.
    df_llm_metrics['datetime'] = pd.to_datetime(df_llm_metrics['timestamp'], errors='coerce', utc=True)
    df_llm_metrics.dropna(subset=['datetime'], inplace=True)
    df_llm_metrics.set_index('datetime', inplace=True)
    df_llm_metrics.sort_index(inplace=True)
    print("✅ LLM-Augmented metrics loaded.")
except Exception as e:
    print(f"❌ Could not load LLM-Augmented dataset. Error: {e}")
    df_llm_metrics = pd.DataFrame()


print("\nLoading tweets dataset...")
try:
    path
    kaggle_csv_path = os.path.join(path, "mbsa.csv")
    df_tweets = pd.read_csv(kaggle_csv_path, usecols=['Date', 'text'], nrows=100000000)
    df_tweets['datetime'] = pd.to_datetime(df_tweets['Date'], errors='coerce', utc=True)
    df_tweets.dropna(subset=['datetime'], inplace=True)
    df_tweets.set_index('datetime', inplace=True)
    df_tweets.sort_index(inplace=True)
    print("✅ Tweets loaded.")
except NameError:
    print("❌ Error: The 'path' variable is not defined.")
    df_tweets = pd.DataFrame()


print("\nLoading news dataset...")
ds_news = load_dataset("edaschau/bitcoin_news")
df_news = ds_news['train'].to_pandas()
df_news['datetime'] = pd.to_datetime(df_news['date_time'], errors='coerce', utc=True)
df_news.dropna(subset=['datetime'], inplace=True)
df_news.set_index('datetime', inplace=True)
df_news.sort_index(inplace=True)
print("✅ News loaded.")



def get_value_for_day(df, day_str, col_name):
    if df.empty or day_str not in df.index:
        return np.nan
    try:
        value = df.loc[day_str, col_name]
        if not pd.api.types.is_scalar(value): value = value.iloc[0]
        return value
    except (KeyError, IndexError, AttributeError):
        return np.nan

daily_dates = pd.date_range(start='2018-02-01', end='2024-5-31', freq='D', tz='UTC')
daily_data_list = []

print(f"\n⚙️  Processing {len(daily_dates)} days to build the final dataset...")
for day in tqdm(daily_dates, desc="Aggregating Daily Data"):
    day_str = day.strftime('%Y-%m-%d')

    news_text_list = df_news.loc[day_str].head(30)['article_text'].tolist() if day_str in df_news.index else []
    tweets_text_list = df_tweets.loc[day_str].head(30)['text'].tolist() if day_str in df_tweets.index else []
    past_prices_df = df_prices[(df_prices.index >= (day - timedelta(days=60))) & (df_prices.index < day)]
    past_60_day_prices = past_prices_df['Close'].tolist()
    future_prices_df = df_prices[(df_prices.index > day) & (df_prices.index <= (day + timedelta(days=10)))]
    future_10_day_prices = future_prices_df['Close'].tolist()
    gold_price = get_value_for_day(df_gold, day_str, 'Close')
    oil_price = get_value_for_day(df_oil, day_str, 'Close')
    past_news_df = df_news[(df_news.index >= (day - timedelta(days=60))) & (df_news.index < day)]
    random_article_text = None
    if not past_news_df.empty:
        random_article_text = past_news_df.sample(n=1)['article_text'].iloc[0]

    hash_rate = get_value_for_day(df_llm_metrics, day_str, 'hash-rate')
    difficulty = get_value_for_day(df_llm_metrics, day_str, 'difficulty')
    transactions = get_value_for_day(df_llm_metrics, day_str, 'n-transactions')
    unique_addresses = get_value_for_day(df_llm_metrics, day_str, 'n-unique-addresses')
    fng_index = get_value_for_day(df_llm_metrics, day_str, 'fng_value')
    cbbi_index = get_value_for_day(df_llm_metrics, day_str, 'cbbi_value')
    llm_sentiment = get_value_for_day(df_llm_metrics, day_str, 'sentiment_class')
    llm_action = get_value_for_day(df_llm_metrics, day_str, 'action_class')
    market_cap = get_value_for_day(df_llm_metrics, day_str, 'market-cap')
    total_bitcoins = get_value_for_day(df_llm_metrics, day_str, 'total-bitcoins')
    estimated_tx_volume = get_value_for_day(df_llm_metrics, day_str, 'estimated-transaction-volume-usd')
    text_cointelegraph = get_value_for_day(df_llm_metrics, day_str, 'cointelegraph')
    text_bitcoin_news = get_value_for_day(df_llm_metrics, day_str, 'bitcoin_news')
    text_reddit = get_value_for_day(df_llm_metrics, day_str, 'reddit')

    daily_data_list.append({
        'date': day.date(),
        'daily_news_full_text': news_text_list,
        'daily_tweets': tweets_text_list,
        'contextual_past_article': random_article_text,
        'gold_close_price': gold_price,
        'oil_close_price': oil_price,
        'btc_price_history_60d': past_60_day_prices,
        'btc_price_target_10d': future_10_day_prices,
        'btc_hash_rate': hash_rate,
        'btc_difficulty': difficulty,
        'btc_transactions': transactions,
        'btc_unique_addresses': unique_addresses,
        'fear_and_greed_index': fng_index,
        'cbbi_index': cbbi_index,
        'llm_sentiment_class': llm_sentiment,
        'btc_market_cap': market_cap,
        'btc_total_supply': total_bitcoins,
        'btc_estimated_tx_volume_usd': estimated_tx_volume,
        'text_cointelegraph': text_cointelegraph,
        'text_bitcoin_news': text_bitcoin_news,
        'text_reddit_posts': text_reddit,
    })


✅ Price data loaded.

Loading LLM-Augmented Bitcoin dataset from Hugging Face...


/tmp/ipython-input-4011541904.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('BTC-USD GC=F CL=F', start='2014-01-01', end='2023-01-31', progress=False)


✅ LLM-Augmented metrics loaded.

Loading tweets dataset...
✅ Tweets loaded.

Loading news dataset...
✅ News loaded.

⚙️  Processing 2312 days to build the final dataset...


Aggregating Daily Data:   0%|          | 0/2312 [00:00<?, ?it/s]


🔄 Filling missing values...


KeyError: "['llm_action_class'] not in index"

In [16]:


final_dataset = pd.DataFrame(daily_data_list)

print("\n🔄 Filling missing values...")
cols_to_fill_numeric = [
    'gold_close_price', 'oil_close_price', 'btc_hash_rate', 'btc_difficulty',
    'btc_transactions', 'btc_unique_addresses', 'fear_and_greed_index', 'cbbi_index',
    'btc_market_cap', 'btc_total_supply', 'btc_estimated_tx_volume_usd'
]
final_dataset[cols_to_fill_numeric] = final_dataset[cols_to_fill_numeric].ffill()
cols_to_fill_text = [
    'llm_sentiment_class', 'text_cointelegraph',
    'text_bitcoin_news', 'text_reddit_posts'
]
final_dataset[cols_to_fill_text] = final_dataset[cols_to_fill_text].fillna('no_data')
print("✅ Missing values filled.")

print("\n\n✅ Final rich dataset created successfully!")
display(final_dataset.head())


🔄 Filling missing values...
✅ Missing values filled.


✅ Final rich dataset created successfully!


,date,daily_news_full_text,daily_tweets,contextual_past_article,gold_close_price,oil_close_price,btc_price_history_60d,btc_price_target_10d,btc_hash_rate,btc_difficulty,...,btc_unique_addresses,fear_and_greed_index,cbbi_index,llm_sentiment_class,btc_market_cap,btc_total_supply,btc_estimated_tx_volume_usd,text_cointelegraph,text_bitcoin_news,text_reddit_posts
0,2018-02-01,[Tax reform has had a major impact on the curr...,"[Be judicious, buy your bitcoin at https://Bit...",JPMorgan Chief Executive Jamie Dimon regrets c...,1344.300049,65.800003,"[11323.2001953125, 11657.2001953125, 11916.700...","[8830.75, 9174.91015625, 8277.009765625, 6955....",2.070395e+07,2.603077e+12,...,591551.0,0.30,0.77,negative,1.972803e+11,16832075.0,2.614261e+09,"[[15052, 'regulated-government-issued-cryptos-...","[[106827, '2018-02-01 23:55:06', 'Senate of Ni...","[['u/Beastly4k', 'OmiseGO primed for 2018! A b..."
1,2018-02-02,[E-commerce veteran and cloud-computing expert...,"[Bitcoin @ £6,109.31 | Ethereum @ £665.00 | Li...",Chipmakers Himax (NASDAQ: HIMX) and Qualcomm (...,1333.699951,65.449997,"[11657.2001953125, 11916.7001953125, 14291.5, ...","[9174.91015625, 8277.009765625, 6955.270019531...",2.070395e+07,2.603077e+12,...,591551.0,0.15,0.76,negative,1.972803e+11,16832075.0,2.614261e+09,no_data,no_data,no_data
2,2018-02-03,[National Oilwell Varco(NYSE: NOV)will report ...,[#ada Ada bull run has begun. Going to $2.00 o...,By Tommy Wilkes and Hideyuki Sano LONDON/TOKYO...,1333.699951,65.449997,"[11916.7001953125, 14291.5, 17899.69921875, 16...","[8277.009765625, 6955.27001953125, 7754.0, 762...",2.044515e+07,2.603077e+12,...,436197.0,0.40,0.75,negative,1.972803e+11,16832075.0,1.469365e+09,"[[15088, 'cryptocurrency-markets-move-back-int...","[[107636, '2018-02-03 23:30:30', 'Bitcoin.com’...","[['u/01-23_45', 'A Theory of Tether', 42, '201..."
3,2018-02-04,[If you file for Social Security benefits at a...,[2018-02-04 00:46:52 Bitcoin price currently a...,We hear a lot about the importance of saving f...,1333.699951,65.449997,"[14291.5, 17899.69921875, 16569.400390625, 151...","[6955.27001953125, 7754.0, 7621.2998046875, 82...",2.044515e+07,2.603077e+12,...,436197.0,0.24,0.75,neutral,1.972803e+11,16832075.0,1.469365e+09,no_data,no_data,no_data
4,2018-02-05,[Bitcoin Cash Falls Again The weekend failed t...,[I have a close family member (naming who name...,"Stocks (^DJI,^GSPC,^IXIC) are clawing higher a...",1333.000000,64.150002,"[17899.69921875, 16569.400390625, 15178.200195...","[7754.0, 7621.2998046875, 8265.58984375, 8736....",2.393894e+07,2.603077e+12,...,486554.0,0.11,0.71,neutral,1.972803e+11,16832075.0,1.950771e+09,"[[15118, 'traditional-markets-nosedive-followi...","[[108411, '2018-02-05 23:55:27', 'New York Wan...","[['u/Timelapze', 'First commercial of the supe..."


In [ ]:
from datasets import load_dataset
import pandas as pd

# Define the repository ID and the specific file path
repo_id = "danilocorsi/LLMs-Sentiment-Augmented-Bitcoin-Dataset"
file_path = "annotated/merged_daily_mistral-nemo_12b-instruct-2407-q5_K_S_opinion.csv"

# Load the specific CSV file
# We pass the file path in the 'data_files' argument.
# The key 'train' is the name we give to this data split.
ds = load_dataset(repo_id, data_files={'train': file_path})

print("✅ File loaded successfully!")
print(ds)

In [26]:
len(df_llm_metrics["reddit"])

25718

In [7]:
!pip install yfinance

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 105.6 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=84f5bdb2fa4f3df461595e22e303916fe7d69c3879f6e949d42ce3456f19cb51
  Stored in directory: /root/.cache/pip/wheels/42/d6/84/bf57a755f4569494cd00de4bb46ef064874823f4d19c82e960
  Created wheel for peewee: filename=peewee-3.18.2-cp311-cp311-linux_x86_64.whl size=886654 sha256=86e7c38be3c0b7bbcd492482aa756de57b8205888e28404c737117c316022a78
  Stored in directory: /root/.cache/pip/wheels/28/84/61/7

In [100]:
final_dataset["daily_news_full_text"][40]

['Major U.S. and European indexes struggled on Monday as Wall Street slogs through an earnings seasons that has largely disappointed. CNBC "Fast Money" trader Brian Kelly said that looking for growth in the current market is "dangerous." Opportunities can still be found, however, and CNBC\'s "Fast Money" traders discussed the sectors they believe hold the best growth prospects. Economic trends point to strong growth for companies in the housing sector, trader Pete Najarian said. Some home builders have "plenty of room to the upside still," he argued. "I look at the way the economy is improving, I look at the lower rates and I look at the cash positions of some of these," Najarian said. Read More Why HK property won\'t bring down its economy He eyed home construction materials producer Masco (NYSE: MAS) in particular. It traded to about $26 per share on Monday. Trader Karen Finerman looked to a newer investment-drones. She has yet to take on any exposure to the sector. However, she says

In [52]:
import pandas as pd
from datasets import Dataset


print("Starting transformation with updated instruction format...")

llm_finetuning_data = []

for index, row in final_dataset.iterrows():

    price_history_str = ", ".join([f"{p:.2f}" for p in row['btc_price_history_60d']])
    news_str = (str(row['daily_news_full_text']) + '...') if row['daily_news_full_text'] else "No specific news available."

    instruction = (
        "You are an expert financial analyst. Your primary task is to predict the next 10 days of Bitcoin prices. "
        "Analyze the provided news and historical price data to make your forecast.\n\n"
        f"**Today's Key News:**\n{news_str}\n\n"
        f"**Last 60 Days of Bitcoin's Closing Prices:**\n[{price_history_str}]\n\n"
        "Use the additional daily data provided in the input below for crucial context."
    )


    gold_price = f"${row['gold_close_price']:.2f}" if pd.notna(row['gold_close_price']) else "N/A"
    oil_price = f"${row['oil_close_price']:.2f}" if pd.notna(row['oil_close_price']) else "N/A"
    market_cap = f"${row['btc_market_cap']:,.0f}" if pd.notna(row['btc_market_cap']) else "N/A"
    fear_greed = f"{row['fear_and_greed_index']:.2f}" if pd.notna(row['fear_and_greed_index']) else "N/A"

    input_text = f"""
Daily Context for Date: {row['date']}

**Financial & Commodity Data:**
- Gold Closing Price: {gold_price}
- Crude Oil Closing Price: {oil_price}

**Bitcoin Market & On-Chain Metrics:**
- Market Capitalization: {market_cap}
- Hash Rate: {row['btc_hash_rate']}
- Transaction Count: {row['btc_transactions']}
- Unique Addresses: {row['btc_unique_addresses']}

**Social & AI Sentiment:**
- Fear & Greed Index: {fear_greed}

**Other Textual Data:**
- Daily Tweets (Sample): {str(row['daily_tweets']) + '...' if row['daily_tweets'] else 'N/A'}
- Contextual Past News Article: {str(row['contextual_past_article']) + '...' if row['contextual_past_article'] else 'N/A'}
- Reddit Posts (Sample): {str(row['text_reddit_posts']) + '...' if row['text_reddit_posts'] else 'N/A'}

Based on the news and historical prices provided in the instruction and this contextual data, predict the next 10 days of Bitcoin's closing price.
"""

    output_text = ", ".join([f"{p:.2f}" for p in row['btc_price_target_10d']])

    llm_finetuning_data.append({
        'instruction': instruction.strip(),
        'input': input_text.strip(),
        'output': output_text
    })


df_finetuning_full = pd.DataFrame(llm_finetuning_data)
hf_finetuning_dataset_full = Dataset.from_pandas(df_finetuning)

print("\n✅ Dataset transformation complete with the new instruction format!")

Starting transformation with updated instruction format...

✅ Dataset transformation complete with the new instruction format!


In [56]:
df_finetuning_full = pd.DataFrame(llm_finetuning_data)
hf_finetuning_dataset_full = Dataset.from_pandas(df_finetuning_full)


In [54]:
df_finetuning_full["instruction"][1001]

'You are an expert financial analyst. Your primary task is to predict the next 10 days of Bitcoin prices. Analyze the provided news and historical price data to make your forecast.\n\n**Today\'s Key News:**\n[\'The major Asia-Pacific stock indexes were lower across the board on Thursday with China bucking the trend to move modestly higher. The weakness in the East wasn’t nearly as bad as the routs in the United States and Europe the previous sessions, as Asia’s brighter economic outlook offset investor worries about fresh COVID-19 lockdowns in Europe. In the cash market on Thursday, Japan’s Nikkei 225 Index settled at 23331.94, down 86.57 or -0.37%. Hong Kong’s Hang Seng Index finished at 24550.17, down 158.63 or -0.64% and South Korea’s KOSPI Index closed at 2326.67, down 18.59 or -0.79%. Bank of Japan Trims Growth, Price Forecasts; Keeps Policy Steady The Bank of Japan on Thursday trimmed its economic and price forecasts for the current fiscal year ending in March 2021, heightening e

In [57]:
# 1. Define the name for your dataset on the Hub
# ❗️ IMPORTANT: Replace "your-username" with your actual Hugging Face username.
repo_id = "tahamajs/bitcoin-llm-finetuning-dataset_all"

# 2. Push the dataset to the Hub
print(f"Uploading dataset to '{repo_id}'...")
hf_finetuning_dataset_full.push_to_hub(repo_id)

print(f"\n🚀 Success! View your dataset at: https://huggingface.co/datasets/{repo_id}")

Uploading dataset to 'tahamajs/bitcoin-llm-finetuning-dataset_all'...


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   4%|3         | 4.78MB /  129MB            

README.md:   0%|          | 0.00/353 [00:00<?, ?B/s]


🚀 Success! View your dataset at: https://huggingface.co/datasets/tahamajs/bitcoin-llm-finetuning-dataset_all


In [19]:
from datasets import Dataset
import yaml

hf_dataset = Dataset.from_pandas(final_dataset)

metadata = {
    'license': 'other',
    'language': 'en',
    'pretty_name': 'Daily Bitcoin Raw News, Tweets, and Financial Indicators',
    'tags': ['time-series', 'bitcoin', 'sentiment-analysis', 'finance', 'news', 'tweets', 'multimodal']
}

readme_content = f"""
---
{yaml.dump(metadata, default_flow_style=False).strip()}
---

# Daily Bitcoin Raw News, Tweets, and Financial Indicators

## Dataset Description

This dataset provides a daily, multimodal view of the Bitcoin ecosystem from **2015-01-01 to 2022-12-31**. It is designed for time-series forecasting and sentiment analysis tasks related to cryptocurrency price movements. Each row corresponds to a single day and contains a rich set of features, including:
- Up to 30 raw, **full-text news articles** related to Bitcoin.
- Up to 30 raw, full-text tweets mentioning Bitcoin.
- The full text of a single random news article from the preceding 60 days for historical context.
- The daily closing prices for Gold and Crude Oil.
- A 60-day historical window of Bitcoin's closing price.
- A 10-day future window of Bitcoin's closing price.

This dataset is designed for tasks such as time-series forecasting, sentiment analysis, and multimodal machine learning to predict cryptocurrency price movements.

## Data Fields

The dataset includes the following columns:

- `date` (date): The specific date for the data record in YYYY-MM-DD format.
- `daily_news_full_text` (list of strings): A collection of up to 30 raw, **full-text news articles** published on the given `date`.
- `daily_tweets` (list of strings): A collection of up to 30 raw, full-text tweets mentioning Bitcoin posted on the given `date`.
- `contextual_past_article` (string): The complete, raw text of a single, randomly selected news article from the 60 days prior to the `date`, intended to provide historical context.
- `gold_close_price` (float): The closing price of Gold futures (GC=F) for the day. Non-trading days are forward-filled with the last available price.
- `oil_close_price` (float): The closing price of WTI Crude Oil futures (CL=F) for the day. Non-trading days are forward-filled with the last available price.
- `btc_price_history_60d` (list of floats): A list containing the daily closing prices of Bitcoin (BTC-USD) for the 60 days *leading up to* the `date`. This serves as the primary historical feature set for time-series models.
- `btc_price_target_10d` (list of floats): A list containing the daily closing prices of Bitcoin (BTC-USD) for the 10 days *following* the `date`. This is the intended **prediction target** for forecasting models.

## Curation and Limitations

- **Raw Text:** The `daily_news_full_text` and `daily_tweets` fields contain unprocessed text. They have **not been summarized** and may require cleaning or embedding for use in models.
- **Data Sources:**
    - **Prices:** Yahoo Finance (`yfinance` library) for BTC-USD, GC=F, and CL=F.
    - **News:** `edaschau/bitcoin_news` dataset on the Hugging Face Hub.
    - **Tweets:** A user-provided `mbsa.csv` file.
- **Missing Data:** Gaps in Gold and Oil prices due to market closures (weekends, holidays) have been filled using a forward-fill (`ffill`) strategy.

## Potential Use Cases

- **Multimodal Price Prediction:** Build models that use text sentiment, financial indicators, and price history to forecast future Bitcoin prices.
- **Sentiment-Driven Trading Strategy:** Analyze the sentiment of daily news and tweets and correlate it with market movements.
- **Economic Indicator Analysis:** Study the correlation between Bitcoin's price and traditional commodities like Gold and Oil.
"""

with open("README.md", "w") as f:
    f.write(readme_content)

print("✅ README.md file has been completely updated.")

✅ README.md file has been completely updated.


In [20]:
from datasets import Dataset

repo_id = "tahamajs/bitcoin-daily-raw-news-tweets-financials"

hf_dataset = Dataset.from_pandas(final_dataset)


print(f"Uploading dataset to '{repo_id}'...")

hf_dataset.push_to_hub(
    repo_id,
    commit_message="Initial upload of the complete, curated dataset"
)

print(f"\n🚀 Successfully uploaded! View your dataset at: https://huggingface.co/datasets/{repo_id}")

Uploading dataset to 'tahamajs/bitcoin-daily-raw-news-tweets-financials'...


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   3%|3         | 4.79MB /  158MB            

README.md:   0%|          | 0.00/609 [00:00<?, ?B/s]


🚀 Successfully uploaded! View your dataset at: https://huggingface.co/datasets/tahamajs/bitcoin-daily-raw-news-tweets-financials


In [21]:
from datasets import load_dataset
import pandas as pd

repo_id = "danilocorsi/LLMs-Sentiment-Augmented-Bitcoin-Dataset"
file_path = "annotated/merged_daily_mistral-nemo_12b-instruct-2407-q5_K_S_opinion.csv"
ds = load_dataset(repo_id, data_files={'train': file_path})

print("✅ File loaded successfully!")
print(ds)

Generating train split: 0 examples [00:00, ? examples/s]

✅ File loaded successfully!
DatasetDict({
    train: Dataset({
        features: ['timestamp', 'open', 'close', 'high', 'low', 'volume', 'blocks-size', 'avg-block-size', 'n-transactions-total', 'n-transactions-per-block', 'hash-rate', 'difficulty', 'miners-revenue', 'transaction-fees-usd', 'n-unique-addresses', 'n-transactions', 'estimated-transaction-volume-usd', 'total-bitcoins', 'market-cap', 'fng_value', 'fng_value_classification', 'fng_sentiment', 'cbbi_value', 'cbbi_sentiment', 'cointelegraph', 'bitcoin_news', 'reddit', 'avg_current_price', 'avg_next_price', 'pct_price_change', 'trend', 'reasoning_text', 'sentiment_class', 'action_class', 'action_score'],
        num_rows: 2338
    })
})


In [83]:
final_dataset


,date,news_headlines,tweets,gold_price,oil_price,past_60_day_prices,future_10_day_prices
0,2015-01-01,[Tokyo police figure Mt. Gox bitcoin heist was...,"[buysellbitco.in #bitcoin price in INR, Buy : ...",NaN,NaN,"[325.8919982910156, 327.5539855957031, 330.492...","[315.0320129394531, 281.0820007324219, 264.195..."
1,2015-01-02,[Raise a glass and toast the U.S. Economy in 2...,[LIVE: Profit = $713.51 (27.63 %). BUY B8.19 @...,1186.000000,52.689999,"[327.5539855957031, 330.49200439453125, 339.48...","[281.0820007324219, 264.19500732421875, 274.47..."
2,2015-01-03,[10 pieces of tech speak we should trash in 20...,[Current price: 195.69£ $BTCGBP $btc #bitcoin ...,NaN,NaN,"[330.49200439453125, 339.4859924316406, 349.29...","[264.19500732421875, 274.4739990234375, 286.18..."
3,2015-01-04,[Bitcoin Is Getting Smoked To Start The New Ye...,[At the rate bitcoin has fallen in the last fe...,NaN,NaN,"[339.4859924316406, 349.2900085449219, 342.415...","[274.4739990234375, 286.1889953613281, 294.337..."
4,2015-01-05,[SpaceBIT Introduces First Ever Space Banking ...,"[In the last 10 mins, there were arb opps span...",1203.900024,50.040001,"[349.2900085449219, 342.4150085449219, 345.488...","[286.1889953613281, 294.3370056152344, 283.348..."
...,...,...,...,...,...,...,...
2917,2022-12-27,[FOREX-Dollar retreats as risk appetite improv...,[],1814.800049,79.529999,"[20595.3515625, 20818.4765625, 20635.603515625...","[16552.572265625, 16642.341796875, 16602.58593..."
2918,2022-12-28,[First Mover Asia: Bitcoin Drifts Lower as Cry...,[],1807.900024,78.959999,"[20818.4765625, 20635.603515625, 20495.7734375...","[16642.341796875, 16602.5859375, 16547.4960937..."
2919,2022-12-29,[First Mover Asia: Bitcoin Retreats Further In...,[],1819.500000,78.400002,"[20635.603515625, 20495.7734375, 20485.2734375...","[16602.5859375, 16547.49609375, 16625.08007812..."
2920,2022-12-30,[Jazz Pharmaceuticals plc (NASDAQ:JAZZ) Q3 202...,[],1819.699951,80.260002,"[20495.7734375, 20485.2734375, 20159.50390625,...","[16547.49609375, 16625.080078125, 16688.470703..."


In [16]:
df_tweets

""
